In [5]:
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from bayes_opt import BayesianOptimization

images = []
labels = []
for image_file_name in os.listdir('dog_cat_images'):
    if image_file_name == '.ipynb_checkpoints': # ignore .ipynb_checkpoints 
        continue 
    if image_file_name == '.DS_Store': # ignore .DS_Store
        continue
    image_path = os.path.join('dog_cat_images', image_file_name)
    #print(f'image_path: {image_path}')
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))
    images.append(img)
    if image_file_name[:3] == 'dog':
        #print(f'label: {0}')
        labels.append(0)
    else:
        #print(f'label: {1}')
        labels.append(1)
images = np.array(images)
labels = np.array(labels)
print(f'images.shape: {images.shape}')
print(f'labels.shape: {labels.shape}')

images.shape: (400, 128, 128, 3)
labels.shape: (400,)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, shuffle=True)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, shuffle=True)


In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │       802,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 896,641 (3.42 MB)

 Trainable params: 896,641 (3.42 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# 이전 학습
history_before_optimization = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# 이전 학습 결과 출력
print("이전 학습 결과:")
print("훈련 정확도:", history_before_optimization.history['accuracy'][-1])
print("검증 정확도:", history_before_optimization.history['val_accuracy'][-1])


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 166ms/step - accuracy: 0.4278 - loss: 17.5508 - val_accuracy: 0.4750 - val_loss: 1.1078
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 159ms/step - accuracy: 0.5373 - loss: 0.8114 - val_accuracy: 0.4250 - val_loss: 0.7689
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 159ms/step - accuracy: 0.5987 - loss: 0.6697 - val_accuracy: 0.5000 - val_loss: 0.7264
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 160ms/step - accuracy: 0.6467 - loss: 0.6117 - val_accuracy: 0.4750 - val_loss: 0.7012
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 159ms/step - accuracy: 0.7694 - loss: 0.5015 - val_accuracy: 0.5250 - val_loss: 0.7414
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 159ms/step - accuracy: 0.8780 - loss: 0.3864 - val_accuracy: 0.5000 - val_loss: 0.8883
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 159ms/step - accuracy: 0.8769 - loss: 0.2963 - val_accuracy: 0.6500 - val_loss: 0.7294
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 160ms/step - accuracy: 0.9011 - loss: 0.2550 - val_accuracy: 0

In [9]:
# BayesianOptimization
def train_evaluate(learning_rate, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=10, batch_size=int(batch_size), validation_data=(X_val, y_val), verbose=0)
    _, accuracy = model.evaluate(X_val, y_val, verbose=0)
    return accuracy

optimizer = BayesianOptimization(
    f=train_evaluate,
    pbounds={'learning_rate': (1e-5, 1e-2), 'batch_size': (16, 100)},
    random_state=42,
)

optimizer.maximize(init_points=5, n_iter=10)

print("최적화된 하이퍼파라미터:", optimizer.max)


|   iter    |  target   | batch_... | learni... |
-------------------------------------------------
| 1         | 0.55      | 47.46     | 0.009508  |
| 2         | 0.525     | 77.49     | 0.005991  |
| 3         | 0.45      | 29.11     | 0.001568  |
| 4         | 0.375     | 20.88     | 0.008663  |
| 5         | 0.625     | 66.49     | 0.007084  |
| 6         | 0.55      | 58.64     | 0.004163  |
| 7         | 0.525     | 65.41     | 0.005498  |
| 8         | 0.475     | 67.11     | 0.00715   |
| 9         | 0.65      | 16.47     | 0.004643  |
| 10        | 0.475     | 72.07     | 0.006425  |
| 11        | 0.4       | 16.47     | 0.005048  |
| 12        | 0.475     | 53.11     | 0.008245  |
| 13        | 0.4       | 17.56     | 0.004389  |
| 14        | 0.425     | 49.19     | 0.004879  |
| 15        | 0.425     | 73.98     | 0.001233  |
최적화된 하이퍼파라미터: {'target': 0.6499999761581421, 'params': {'batch_size': 16.47405710116639, 'learning_rate': 0.004643160820803875}}


In [14]:
# 최적화된 하이퍼파라미터로 모델 재구성 및 훈련
best_learning_rate = optimizer.max['params']['learning_rate']
best_batch_size = optimizer.max['params']['batch_size']

model_optimized = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_optimized.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_learning_rate),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

history_after_optimization = model_optimized.fit(X_train, y_train, epochs=10, batch_size=int(best_batch_size), validation_data=(X_val, y_val))



print("베이지안 최적화 이후 모델의 정확도:")
_, accuracy_after_optimization = model_optimized.evaluate(X_val, y_val)
print("검증 정확도:", accuracy_after_optimization)


print("정확도 변화:")
accuracy_change = accuracy_after_optimization - history_before_optimization.history['val_accuracy'][-1]
print("이전 학습에서 최적화 후 정확도 변화:", accuracy_change)


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - accuracy: 0.5883 - loss: 57.9710 - val_accuracy: 0.4750 - val_loss: 0.7097
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - accuracy: 0.5107 - loss: 0.7026 - val_accuracy: 0.7000 - val_loss: 0.6593
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - accuracy: 0.5831 - loss: 0.6960 - val_accuracy: 0.5500 - val_loss: 0.6850
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - accuracy: 0.5033 - loss: 0.6895 - val_accuracy: 0.6000 - val_loss: 0.6843
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - accuracy: 0.7326 - loss: 0.5975 - val_accuracy: 0.6000 - val_loss: 0.9078
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - accuracy: 0.8409 - loss: 0.4697 - val_accuracy: 0.5500 - val_loss: 0.8052
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.8168 - loss: 0.4455 - val_accuracy: 0.6250 - val_loss: 1.0233
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.8816 - loss: 0.3422 - val_accuracy: 0.6000 - 

In [12]:
print("최적 배치 크기: ", int(best_batch_size))

최적 배치 크기:  16
